# This is an implementation for computing Collision Free Regions in Configuration Space using IRIS

[IRIS in Configuration Space](https://drake.mit.edu/doxygen_cxx/group__geometry__optimization.html#ga3a51e0fec449a0abcf498f78a2a390a8)


This notebook contains code for creating, visualizing and saving such regions.

In [ ]:
import sys
import os

if 'google.colab' in sys.modules:
  print("You are in colab")
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/gcs-motion-planning-main/jupyter_notebooks')
  !pip install pyngrok drake

import numpy as np

from pyngrok import ngrok

from pydrake.geometry import ( StartMeshcat )
from pydrake.geometry.optimization import (SaveIrisRegionsYamlFile)
from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.visualization import AddDefaultVisualization

# add parent folders to path
sys.path.append(os.path.abspath('..'))

# Update the import
from src.manipulation import running_as_notebook
from src.auxiliar.auxiliar_functions import (VisualizeConnectivity, LoadRobot)
from src.auxiliar.homemade_gcs import GcsTrajOpt
from src.auxiliar.iris_options import create_iris_options
from src.auxiliar.joint_teleop import JointTeleop
from src.auxiliar.helper import VisualizeRegions

In [ ]:
# Start the visualizer (only run this once).
meshcat = StartMeshcat()
if 'google.colab' in sys.modules:
  # Create a tunnel to the Meshcat server
  ngrok.set_auth_token("2rdws4UBYTqZ1NiiyEbJcWRnByV_7D9Jg8K3YVyZzbFMN1ysg")
  public_url = ngrok.connect(7000, "http")
  print(f"Meshcat is accessible at: {public_url}")

# Setup



In [ ]:
# Get the absolute path of the current working directory
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_folder = os.path.join(parent_dir, 'data')
iris_filename = os.path.join(data_folder, 'my_iris.yaml')  # The file to save the regions to
iris_regions = dict() # The regions that we will compute
q = []
iris_options, config = create_iris_options() # Create the options for the iris algorithm

## Run IRIS on manually-specified seeds

In [ ]:
print(config["iris_regions"].keys())
iris_regions = config["iris_regions"]
VisualizeRegions(meshcat, iris_regions, config)

In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
LoadRobot(plant)
plant.Finalize()
AddDefaultVisualization(builder, meshcat)
diagram = builder.Build()

diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_context)

diagram.ForcedPublish(diagram_context)

## Joint Teleop

In [ ]:
# clear the meshcat visualizer
meshcat.Delete()
meshcat.DeleteAddedControls()
seeds_joint_teleop = []

JointTeleop(meshcat, seeds_joint_teleop, config["iris_regions"], config, q)

## Visualize the current IRIS region connectivity

For planning trajectories, it's important to have some non-empty intersection between the convex sets. Here is a simple utility the visualization the connectivity as a graph.

In [ ]:
VisualizeConnectivity(config['iris_regions'])

In [ ]:
print(seeds_joint_teleop)

## Plan trajectories with GCS Trajectory Optimization

In [ ]:
# clear the meshcat visualizer
meshcat.Delete()
meshcat.DeleteAddedControls()

assert (
    seeds_joint_teleop
), "The examples here use the 'manually-specified seeds' from the section above. Please run that section first, or populate your own start and end configurations."

GcsTrajOpt(seeds_joint_teleop[0], seeds_joint_teleop[1],config["iris_regions"], meshcat)

# Save your regions back to disk

Note that by default, this notebook will autosave a backup after making each region.

In [ ]:
if running_as_notebook:
    SaveIrisRegionsYamlFile(iris_filename, iris_regions)